# Relative Transfer Function modeling for supervised source localization

In this notebook, we attempt to reproduce the results of the following paper : *[Relative Transfer Function modeling for supervised source localization](https://users.math.yale.edu/rt294/WASPAA_Laufer2013.pdf)*. The authors propose an approach based on **manifold learning** to tackle the limitations related to high reverabaration levels and large number of microphones required when dealing with the sound source localization problem. The proposed algorithm can localize a source in a 3D room using only one pair of microphones and by fixing the distance from the microphones to the source. The algorithm works as follow :

*Training stage*

* **Define the feature vector of source $i$ as:  $T_{i}$ = $\displaystyle\frac{H_{i1}(e^{i\omega_{r}})}{H_{i2}(e^{i\omega_{r}})}$, where  $\omega_{r} =
\frac{2\pi r}{D} ; r = 0,...,D-1$ denotes a discrete frequency index and the transfer functions are between the source $i$ and the 2 microphones. **

* **Compute the affinity matrix W ($m*m$) between the $m$ training samples defined for the selected feature vectors.**

* **Let $\lambda_{j}$ and $\varphi_{j}$, $j = 0, 1, ..., m-1$  be the eigenvalues and eigenvectors of the affinity matrix W. **

*Test stage*

* **Given new set of $M$ test positions with unknown locations, construct an asymmetric affinity matrix $A$ between the feature vectors of the entire set (training and test) and the training set**

* **Normalize assymetric kernel $A$ as : $N = AS^{-\frac{1}{2}}$ where $S = diag\{A^{T}A\textbf{1}\} $. **

* **Compute the left-singular vectors $\psi_{j}$ of $N$ as $\displaystyle\psi_{j} = \frac{1}{\sqrt{\lambda_{j}}} N \varphi_{j} $.**

* **Reconstruct an embedding of the measurements onto the space spanned by the $d$ left singular vectors in correspondence with the dimensions of the parameter space ($d = 2$ in our case) :<br> $\displaystyle\Psi : \textbf{T}_{i} \rightarrow [ \psi_{1}^{i}, \psi_{2}^{i} ]^{T}$, where $\psi_{1}^{i}$ denotes the $ith$ entry of $\psi_{1}$. **
 
* **Estimate the test location parameters as follow : $\displaystyle\hat\theta_{i} = \sum_{j}\gamma_{j}(T_{i})\bar\theta_{j}$, where $\gamma_{j}(T_{i})$ are the interpolation coefficients, $T_{i}$ is the test feature vector and  $\bar\theta_{j}$ is the known parameter locations (azimuth and elevation angle) of training source $j$**

In [1]:
import pyroomacoustics as pra
import numpy as np
import matplotlib.pyplot as plt
import math
import scipy
from tqdm import tqdm
from script_rtf import *

**Define some constants**

Modifications need to be made also in the script `helpers_rtf.py`

In [2]:
fs = 8000 # sampling frequencey
m = 50 # number of sources in training stage
M = 49 # number of sources in test stage
L = 5 # number of perturbations around a given source 
d = 2 # degrees of freedom (in this case azimuth and elevation angle)
epsilon = 1e20 # scaling factor
eps_var = 1e-5 # sclaing factor
k = 3 # number of neigherest neighbour selected

### Training stage

**Compute _*affinity matrix*_ $W$ **

In [3]:
cov_matrices = compute_covMatrices(postions_Of_perturbations, m=m, L=L)
W = compute_affinity_matrix(cov_matrices, training_Set_xyz, m=m)
eig_vals, eig_vects = np.linalg.eig(W)

100%|██████████| 50/50 [00:21<00:00,  2.36it/s]


Remark : *the column `eig_vects[:,i]` is the eigenvector corresponding to the eigenvalue `eig_vals[i]`*

### Test stage

**Compute affinity matrix $A$ between features vectors in entire set (training +test) and training set**

In [4]:
A = W.copy()
A = np.c_[A.T, np.zeros((m,M))].T

In [5]:
for i in tqdm(range(M)):
    for j in range(m):
        test_featVect = feature_Vector_testPos(index_src=i)
        train_featVect = feature_Vector_src(index_src=j)
        inv_cov_mat_j = np.linalg.inv(cov_matrices[j])
        
        nom = (test_featVect - train_featVect) @ inv_cov_mat_j @ (test_featVect - train_featVect).T
        A[i+m,j] = np.exp(-nom/epsilon)

100%|██████████| 49/49 [00:15<00:00,  3.18it/s]


In [6]:
A.shape

(99, 50)

**Compute diagonal matrix $S^{-0.5}=diag\{A^{T}A1\}^{-0.5}$ and $A^{tilde}$**

In [7]:
v = np.matrix(1. /np.sqrt(A.T @ A @ np.ones((m, 1))))
S_sqrt = np.diag(v.A1)
A_tilde = (A @ S_sqrt) 

**Compute the left-singular vectors $\varphi_{j}$ of $A^{tilde}$ as a weighted interpolation of the eigenvectors of $W$**

In [8]:
train_featVects = compute_train_featVects(m=m)
singular_vects = compute_d_left_sing_vect(m+M, eig_vals, eig_vects, A_tilde, d=d) 

**Interpolate the test locations**

In [9]:
thetas_pred = []
thetas_gt = []
for i in range(M):
    theta_i_pred = estimate_theta_i(i, singular_vects, train_featVects, training_Set_spherical)
    thetas_pred.append(theta_i_pred)
    theta_i_gt = np.array([test_Set_spherical[i][1], test_Set_spherical[i][2]])
    thetas_gt.append(theta_i_gt)

**Compute RMSE**

In [10]:
rmse_val = rmse(thetas_gt, thetas_pred)

In [11]:
print('Re-parameterization of the test set yields a mean error of RMSE = {rmse} rad.'.format(rmse= rmse_val))

Re-parameterization of the test set yields a mean error of RMSE = 0.035894932163407646 rad.
